In [10]:
import pathlib

man_dir = pathlib.Path('dataset/MEN')
woman_dir = pathlib.Path('dataset/WOMAN')

man_files = man_dir.glob('*jpg')
woman_files = woman_dir.glob('*jpg')

In [11]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, files, labels, transform=None):
        self._files = files
        self._labels = labels
        self._transform = transform
    
    def __getitem__(self, index):
        image = Image.open(self._files[index])
        if self._transform is not None:
            image = self._transform(image)
        label = self._labels[index]
        return image, label
    
    def __len__(self):
        return len(self._labels)

In [13]:
from torchvision import transforms

def create_image_dataset(file_dir) -> ImageDataset:
    height, width = 224,224

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((height, width))
    ])
    
    labels = []
    files = []
    for i, file in enumerate(file_dir):
        labels.append(0)
        files.append(file)
        
    return ImageDataset(files, labels, transform)

In [14]:
from torch.utils.data import ConcatDataset

man_dataset = create_image_dataset(man_files)
woman_dataset = create_image_dataset(woman_files)

dataset = ConcatDataset([man_dataset, woman_dataset])

In [15]:
from torch.utils.data import random_split

train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

In [16]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

In [17]:
from torch import nn, optim

loss_fn = nn.BCEWithLogitsLoss()

model = nn.Sequential(
    
    nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    
    nn.Flatten(),
    
    nn.Linear(25088, 256),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    
    nn.Linear(256, 1)
)

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
epochs = 2
model.train()

for epoch in range(epochs):

    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        output = model(x_batch).reshape(-1)
        y_batch = y_batch.to(torch.float32)
        loss = loss_fn(output, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if (epoch + 1) % 1 == 0:
        print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


KeyboardInterrupt



In [ ]:
correct = 0
totals = 0
model.eval()

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        logit = model(x_batch)
        labels = torch.argmax(logit, dim=1)
        correct += torch.sum(labels == y_batch).item()
        totals += y_batch.size(0)
print(f"Accuracy: {correct / totals}")

In [ ]:
torch.save(model, 'gender-recognizer.pt')